# Modified 3D PMRNN

## Module Imports

In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import cv2
import os
import Utils
import numpy as np
import matplotlib.pyplot as plt
from CONFIGURATION import CONFIGURATION
from Reconstruction import Reconstruction

2023-06-17 15:00:35.670404: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-17 15:00:37.517395: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 15:00:37.537097: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-17 15:00:37.720970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:00:37.721043: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name:

In [2]:
# ignorable
print("TensorFlow version:", tf.__version__)
print("cv2 version:",cv2.__version__)
print("np version:",np.__version__)

TensorFlow version: 2.5.0
cv2 version: 4.5.5
np version: 1.19.5


In [3]:
# ignorable
import sklearn, matplotlib
print("matplotlib version:",matplotlib.__version__)
print("sklearn version:",sklearn.__version__)

matplotlib version: 3.6.3
sklearn version: 1.2.2


In [4]:
from tensorflow.keras import layers,losses #,Sequential,metrics
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import * 
# from tensorflow.keras.layers import Layer
# from tensorflow.keras import optimizers 
# from tensorflow.keras.optimizers.experimental import SGD
# from tensorflow.image import ssim

## Load new Data

### reading data

In [ ]:
image_data = Utils.read_data("Berea_2d25um_binary.raw")
image_data.shape

### Voxel extraction 

In [ ]:
voxels = Utils.extract_subvolumes(image_data)
voxels.shape

In [ ]:
del image_data

### Train and test data extraction

In [ ]:
X_train = voxels[:-6]
X_test = voxels[-6:]

In [ ]:
del voxels

In [ ]:
len(X_train)

### Save Traited data for uses

In [ ]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)

In [ ]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)

## TensorFlow GPU setup

In [5]:
device = tf.config.list_physical_devices("GPU")
print(device)
tf.config.experimental.set_memory_growth(device[0],True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-06-17 15:01:56.317656: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:01:56.317787: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3070 Laptop GPU computeCapability: 8.6
coreClock: 1.62GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2023-06-17 15:01:56.317889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:01:56.317917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:01:56.317926

In [6]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [7]:
tf.keras.backend.clear_session()

## Model creation

### Build and Compile model

In [ ]:
reconstruction = Reconstruction()
reconstruction.summary()
reconstruction.compile()

## Load Traited data for training (voxels)

In [8]:
X_train=np.load('X_train.npy')
X_train.shape

(58, 250, 256, 256, 1)

### idk

In [9]:
learnedVoxels = []
inferenceVoxels = []
for voxel in X_train:
    learnedVoxels.append(voxel[:-1])
    inferenceVoxels.append(voxel[1:])

# learnedVoxels = np.array(learnedVoxels)
# inferenceVoxels = np.array(inferenceVoxels)

## Training 

### Imports

In [10]:
from mealpy.swarm_based import GWO
from sklearn.preprocessing import LabelEncoder
import time
from datetime import datetime

### Lables

In [18]:
class Timer():
    def __init__(self,title=" ") -> None:
        self.title=title
        self.started=time.time()
        print(f"{self.title} Started at : {datetime.fromtimestamp(self.started)}")
        self.ended=None
    def End(self):
        print(f"{self.title} Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s")
        self.ended=time.time()
    def toDuration(self):
        if self.ended == None:
            return -1
        return self.ended-self.started
    def __str__(self) -> str:
        return f"{self.title} Started at : {datetime.fromtimestamp(self.started)} | Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s"

class TrainingTime(Timer):
    def __init__(self,title=" ") -> None:
        super().__init__(title=title)
        self.samples=[]

    def SampleStarted(self,title=" ")->Timer:
        timer= Timer(title)
        self.samples.append(timer)
        return timer
    

In [11]:
OPT_ENCODER = LabelEncoder()
#OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'])
OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adam'])
trainings_duration=[]

LabelEncoder()

### Solution Decodation function

In [12]:
def decode_solution(solution):
    batch_size = 2**int(solution[0])
    
    learning_rate = solution[1]
    
    opt_int = int(solution[2])
    opt = OPT_ENCODER.inverse_transform([opt_int])[0]
    
    epoch = 10 *int(solution[3])
    
    num_filters = int(solution[4])
    
    latent_space_dim = 2**int(solution[5])
    
    reduced_dim = 2**int(solution[6])
    
    return [batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim,reduced_dim]

### Objective Function

In [19]:
def objective_function(solution):
    tf.keras.backend.clear_session()
    batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim = decode_solution(solution)
    reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
                                     latent_space_dim = latent_space_dim,
                                     reducedDimension = reduced_dim,
                                     num_conv_layers = num_filters,
                                     learning_rate = learning_rate,
                                     batch_size = batch_size,
                                     epochs = epoch,
                                     opt = opt,
                                   )
#     reconstruction.summary()
    reconstruction.compile()
    
    trainingTime=TrainingTime("Epoch Training")
    histories = []
    for i in range(len(learnedVoxels)):
        print(f"currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
        sampleTime = trainingTime.SampleStarted("Sample Training")
        histories.append(reconstruction.train(learnedVoxels[i],inferenceVoxels[i]))
        sampleTime.End()
        
    trainingTime.End()
    trainings_duration.append(trainingTime)
    reconstruction.save(save_folder=f"results/model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}")
    
    return histories[-1].history['loss'][-1]

### Configurations

In [14]:
LB = [3    , 0.001 , 0   , 2      ,  4  ,  6    , 7 ]
UB = [5.99 , 0.01  , 3.99, 4.99  ,  8.99,  8.99 , 9.99 ]

problem_dict = {
    "fit_func": objective_function,
    "lb": LB,
    "ub": UB,
    "minmax": "min",
    "verbose":True,
}
metaheuristic_configuration={
    "epoch":1,
    "pop_size":50
}

### Lunching the metaheuristic

In [15]:
model = GWO.BaseGWO(problem_dict,epoch=metaheuristic_configuration["epoch"],pop_size=metaheuristic_configuration["pop_size"])

Instructions for updating:
Colocations handled automatically by placer.


2023-06-17 15:03:08.361878: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:03:08.361956: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3070 Laptop GPU computeCapability: 8.6
coreClock: 1.62GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2023-06-17 15:03:08.362019: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:03:08.362047: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:03:08.362055

currently working one voxel : 1, voxels left : 57 
Started at : 2023-06-17 15:03:12.850362
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 249 samples


2023-06-17 15:03:16.109113: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:03:16.109174: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3070 Laptop GPU computeCapability: 8.6
coreClock: 1.62GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2023-06-17 15:03:16.109234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:03:16.109261: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 15:03:16.109268

Epoch 1/30


2023-06-17 15:03:26.824526: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-06-17 15:03:28.595175: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2023-06-17 15:03:31.556049: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-17 15:03:31.557750: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2023-06-17 15:03:31.662116: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-17 15:03:31.663555: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-06-17 15:03:32.034952: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

249/249 [==============================] - 20s 82ms/sample - loss: 0.5625 - _calculate_reconstruction_loss: 0.2602 - _calculate_porosity_loss: 0.2253 - _calculate_kl_loss: 75.4577
Epoch 2/30
249/249 [==============================] - 2s 8ms/sample - loss: 0.4801 - _calculate_reconstruction_loss: 0.1982 - _calculate_porosity_loss: 0.2069 - _calculate_kl_loss: 74.5038
Epoch 3/30
249/249 [==============================] - 2s 8ms/sample - loss: 0.4330 - _calculate_reconstruction_loss: 0.1648 - _calculate_porosity_loss: 0.1929 - _calculate_kl_loss: 74.5866
Epoch 4/30
249/249 [==============================] - 2s 9ms/sample - loss: 0.3990 - _calculate_reconstruction_loss: 0.1418 - _calculate_porosity_loss: 0.1810 - _calculate_kl_loss: 75.7438
Epoch 5/30
249/249 [==============================] - 2s 9ms/sample - loss: 0.3705 - _calculate_reconstruction_loss: 0.1249 - _calculate_porosity_loss: 0.1699 - _calculate_kl_loss: 75.1050
Epoch 6/30
249/249 [==============================] - 2s 9ms/sam

2023/06/17 04:33:21 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: Solving single objective optimization problem.


In [ ]:
#model.solve()

In [17]:
print(f"Best solution : {model.solution[0]}")
batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim = decode_solution(model.solution[0])
print(f"Batch size : {batch_size}, learning_rate : {learning_rate}, opt : {opt}, epoch : {epoch}, num_filters : {num_filters}, latent_space_dim : {latent_space_dim}, reduced_dim : {reduced_dim}")
model.history.save_global_objectives_chart(filename="hello/goc")
model.history.save_local_objectives_chart(filename="hello/loc")

model.history.save_global_best_fitness_chart(filename="hello/gbfc")
model.history.save_local_best_fitness_chart(filename="hello/lbfc")

model.history.save_runtime_chart(filename="hello/rtc")

model.history.save_exploration_exploitation_chart(filename="hello/eec")

model.history.save_diversity_chart(filename="hello/dc")

model.history.save_trajectory_chart(list_agent_idx=[3, 5], selected_dimensions=[3], filename="hello/tc")

TypeError: 'NoneType' object is not subscriptable

## Training the model

In [ ]:
histories = []
for i in range(3):
    print(f"currently working one voxel : {i +1}, voxels left : {3-(i+1)} ")
    histories.append(reconstruction.train(learnedVoxels[i],inferenceVoxels[i]))

In [ ]:
saved = Reconstruction.load(save_folder=f"test")

## Testing the model

In [ ]:
first  = X_test[0]
Topredict = first[1:]
test = first[:-1]

In [ ]:
testing = []
inputLayer = first[0]
# for i in range(len(first)-1):
for i in range(10):

    learned = reconstruction.learnedPrior(inputLayer.reshape(1,256,256,1))
    testing.append(reconstruction.Generate.predict([inputLayer.reshape(1,256,256,1),learned],steps=1))
    inputLayer = testing[-1]

In [ ]:
len(testing)

In [ ]:
n = 10

plt.figure(figsize=(20,4))
for i in range(n):
    # original
    ax = plt.subplot(2,n,i+1)
    plt.imshow(Topredict[i].astype("float32"))
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # reconstructed
    ax = plt.subplot(2,n,i+1+n)
    plt.imshow(testing[i].reshape(256,256,1))
    plt.title('reconstructed')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
# reconstruction.Generate.save('reconstruction_10batch_10epochs.h5')

In [ ]:
file_path = "reconstructed.raw"
testing.insert(0, first[0].reshape(1,256,256,1))
print(len(testing))

In [ ]:
final = np.array(testing,np.float32).reshape(250,256,256)
final.shape

In [ ]:
file_path = "my_array.npy"
np.save("file", final)

In [ ]:
final.tofile(file_path)

In [ ]:
Volume = np.fromfile(file_path, dtype=np.uint8)
Volume = Volume.reshape(250,256,256)
Volume.shape